## Generate utility function with higher-order polynomial

In [1]:
import torch 
import torch.nn
import torch.nn.functional as F
import numpy as np
import torch.distributions.log_normal as log_normal
import torch.distributions.uniform as uniform
from sklearn import model_selection
import torch.distributions.bernoulli as bernoulli                   

Utility
$V_j\\
= b_{0j} + b_1 x_1 + b_2 x_2 + b_3 x_3\\
= b_{0j} + b_1 x_1 + (a_0 + a_1z_1 + a_2z_2 + a_3z_3 + a_{12}z_1z_2 + a_{13}z_1z_3 + a_{23}z_2z_3) x_2 \\
= b_{0j} + b_1 x_1 + (a_0 + z^T A_1 + z^T A_2 z) x_2$

In [3]:
def generateZ(N):
    '''
    Generate z=[z0,z1,z2] for N samples (income, full-time, flexible-schedule)
    Return: Z (N, 3)
    '''
    # Define p(z1)
    m_fulltime = bernoulli.Bernoulli(0.5) # full time prob = 0.5
    
    # Define p(z0|z1), p(z2|z1)
    m_inc_fulltime = log_normal.LogNormal(torch.Tensor([np.log(0.5)]), 0.25)
    m_inc = log_normal.LogNormal(torch.Tensor([np.log(0.25)]), 0.2)
    m_flex_fulltime = bernoulli.Bernoulli(0.5)
    m_flex = bernoulli.Bernoulli(0.5) 
    
    z = torch.zeros(N,3) # z = (income, job, flex)
    
    # Draw job 
    z[:,1] = m_fulltime.sample(sample_shape=(N,)) # N by 1
    
    # Given z[:,1]=1: draw z[:,0](income), z[:,2] (flex)
    ind = z[:,1]==1
    ind_sum = ind.sum().item()
    z[ind,0] = m_inc_fulltime.sample(sample_shape=(ind_sum,)).flatten()
    z[ind,2] = m_flex_fulltime.sample(sample_shape=(ind_sum,)).flatten()
    
    # Given z[:,1]=0: draw z[:,0](income), z[:,2] (flex)
    z[~ind,0] = m_inc.sample(sample_shape=(N-ind_sum,)).flatten()
    z[~ind,2] = m_flex.sample(sample_shape=(N-ind_sum,)).flatten()
    return z

In [4]:
def value_of_x(A0, A1, A2, Z):
    '''
    Compute value of time for N persons given person characteristics z (N,D)
    Input:
        A0, A1, A2: 0, 1st and 2nd order interaction coefficients
        Z: person input (N,D)
    Return:
        vox: (N,1)
    '''
    vox = A0 + torch.matmul(Z,A1) + torch.diag(torch.matmul(torch.matmul(Z,A2), Z.transpose(0,1)))
    return vox

In [5]:
def generate_one_x(vot, vowt, min_c = 0.2, max_c = 100, min_t = 5, max_t = 100, min_wt = 5, max_wt = 30):
    '''
    v: value of time ($/min) < 0
    
    return t0,t1,c0,c1 (time and cost for alternative 0 and 1)
    '''
    
    dist_t = uniform.Uniform(min_t, max_t)
    t0 = dist_t.sample().item()
    t1 = dist_t.sample().item()
    
    dist_wt = uniform.Uniform(min_wt, max_wt)
    wt0 = dist_wt.sample().item()
    wt1 = dist_wt.sample().item()
    
    while True:
        c1_c0 = vot*(t1-t0) + vowt*(wt1-wt0) + torch.randn(1)*2 # add random error up and down the line 
        
        if c1_c0 <= 0 and min_c-c1_c0 <= max_c:
            c0 = uniform.Uniform(min_c-c1_c0, max_c).sample().item()
            break
        elif c1_c0 > 0 and min_c <= max_c-c1_c0:
            c0 = uniform.Uniform(min_c, max_c-c1_c0).sample().item()
            break
        else:
            continue

    c1 = c0 + c1_c0
    c1 = c1.item()
    return t0,t1, wt0, wt1, c0,c1

In [6]:
def generateX(vots, vowts):
    '''
    Generate samples x incl intercept: (N,K+1,J) where J = 2
        x[n] = 
            [0, 1
             c0,c1
             t0,t1
             wt_0, wt_1]
        c0, c1: cost in $
        t0, t1: time in minutes 
        wt0, wt1: waiting time in minutes
    Return: 
        X: (N, D, 2)
    '''
    D = 4 # including ASC
    N = vots.size(0)
    X = torch.zeros(N,D,2)
    n = 0
    while (n < N):
        t0, t1, wt0, wt1, c0, c1 = generate_one_x(vots[n], vowts[n])
        X[n] = torch.Tensor([0, 1, c0, c1, t0, t1, wt0, wt1]).reshape(4,2)
        n += 1
    return X

In [7]:
def utility(asc0, asc1, X, vots, vowts):
    '''
    Input:
        X: (N,K+1,J)
        vots: (N)
        vowts: (N)
    Returns:
        V: (N,J)
    '''
    N, K_plus_1, J = X.size()
    K = K_plus_1-1
    V = torch.zeros(N,J)
    for n in range(N): 
        vot_n = vots[n].item()
        vowt_n = vowts[n].item()
        Bn = torch.Tensor([asc0, asc1, -1, -1, vot_n, vot_n, vowt_n, vowt_n]).reshape(K+1,2) #(K+1,J)
        xn = X[n] # (K+1,J)
        V[n] = (Bn*xn).sum(dim=0) # 1 by J
    return V

In [8]:
def generateData(N):
    # generate personal characteristics
    Z = generateZ(N) # N, D

    # compute vot, vowt
    vots = value_of_x(A0_time, A1_time, A2_time, Z) # N
    vowts = value_of_x(A0_wait, A1_wait, A2_wait, Z)
    
    # generate alternative attributes (closer to the decision boundary, which is why we need vots, vowts)
    X = generateX(vots, vowts) # (N, K+1, J) = (N, 4, 2)

    v = utility(asc0, asc1, X, vots, vowts)

    p = torch.softmax(v, dim=1)
    m = torch.distributions.categorical.Categorical(probs=p)
    y = m.sample()
    nll = F.nll_loss(input=torch.log(p),target=y)
    acc = (y==p.argmax(1)).sum().float()/N
    data = {"x": X, 
            "z": Z,
            "y": y, 
            "p": p, 
            "vots": vots, 
            "vowts": vowts, 
            "params": params,
            "nll": nll.item(), 
            "acc": acc.item()}
    
    return data

### Define parameters

In [10]:
# ASC
asc0 = 0
asc1 = -0.1

# 0, 1st and 2nd order effect of z on time and waiting time
A0_time = -0.1
A1_time = torch.Tensor([-0.5, -0.1, 0.05])
A2_time = torch.Tensor(\
                       [[ 0.0000, -0.2000,  0.0500],
                        [ 0.0000,  0.0000,  0.1000],
                        [ 0.0000,  0.0000,  0.0000]])

A0_wait = -0.2
A1_wait = torch.Tensor([-0.8, -0.3, 0.1])
A2_wait = torch.Tensor(\
                       [[ 0.0000, -0.3000,  0.08],
                        [ 0.0000,  0.0000,  0.3000],
                        [ 0.0000,  0.0000,  0.0000]])



In [14]:
params = {
    "asc0": asc0,
    "asc1": asc1,
    "b_time": A0_time,
    "b_time_z1": A1_time[0].item(),
    "b_time_z2": A1_time[1].item(),
    "b_time_z3": A1_time[2].item(),
    "b_time_z1z2": A2_time[0,1].item(),
    "b_time_z1z3": A2_time[0,2].item(),
    "b_time_z2z3": A2_time[1,2].item(),
    "b_wait": A0_wait,
    "b_wait_z1": A1_wait[0].item(),
    "b_wait_z2": A1_wait[1].item(),
    "b_wait_z3": A1_wait[2].item(),
    "b_wait_z1z2": A2_wait[0,1].item(),
    "b_wait_z1z3": A2_wait[0,2].item(),
    "b_wait_z2z3": A2_wait[1,2].item(),
    "A0_time": A0_time,
    "A1_time": A1_time,
    "A2_time": A2_time,
    "A0_wait": A0_wait,
    "A1_wait": A1_wait,
    "A2_wait": A2_wait,
}

In [15]:
params

{'asc0': 0,
 'asc1': -0.1,
 'b_time': -0.1,
 'b_time_z1': -0.5,
 'b_time_z2': -0.10000000149011612,
 'b_time_z3': 0.05000000074505806,
 'b_time_z1z2': -0.20000000298023224,
 'b_time_z1z3': 0.05000000074505806,
 'b_time_z2z3': 0.10000000149011612,
 'b_wait': -0.2,
 'b_wait_z1': -0.800000011920929,
 'b_wait_z2': -0.30000001192092896,
 'b_wait_z3': 0.10000000149011612,
 'b_wait_z1z2': -0.30000001192092896,
 'b_wait_z1z3': 0.07999999821186066,
 'b_wait_z2z3': 0.30000001192092896,
 'A0_time': -0.1,
 'A1_time': tensor([-0.5000, -0.1000,  0.0500]),
 'A2_time': tensor([[ 0.0000, -0.2000,  0.0500],
         [ 0.0000,  0.0000,  0.1000],
         [ 0.0000,  0.0000,  0.0000]]),
 'A0_wait': -0.2,
 'A1_wait': tensor([-0.8000, -0.3000,  0.1000]),
 'A2_wait': tensor([[ 0.0000, -0.3000,  0.0800],
         [ 0.0000,  0.0000,  0.3000],
         [ 0.0000,  0.0000,  0.0000]])}

In [16]:
import pickle
pickle.dump(params, open("toy_data/params.pkl", "wb"))

### Generate data （training 10K)

In [18]:
torch.manual_seed(7)
N_train, N_dev, N_test = 10000, 2000, 2000

data_train = generateData(N_train)
data_dev = generateData(N_dev)
data_test = generateData(N_test)

In [19]:
print (data_train['nll'])
print (data_dev['nll'])
print (data_test['nll'])

0.46460282802581787
0.44120335578918457
0.4465537369251251


In [20]:
data_train.keys()

dict_keys(['x', 'z', 'y', 'p', 'vots', 'vowts', 'params', 'nll', 'acc'])

In [21]:
data = {"train": data_train, 'dev': data_dev, "test": data_test}

In [18]:
import pickle
pickle.dump(data_train, open("toy_data/train_10k.pkl","wb"))
pickle.dump(data_dev, open("toy_data/dev_10k.pkl","wb"))
pickle.dump(data_test, open("toy_data/test_10k.pkl","wb"))
pickle.dump(data, open("toy_data/data_10k.pkl","wb"))

### Generate data（training 100K)

In [22]:
torch.manual_seed(7)
N_train, N_dev, N_test = 100000, 2000, 2000

data_train = generateData(N_train)
data_dev = generateData(N_dev)
data_test = generateData(N_test)

In [23]:
print (data_train['nll'])
print (data_dev['nll'])
print (data_test['nll'])

0.4619007706642151
0.44855937361717224
0.4844731390476227


In [24]:
data = {"train": data_train, 'dev': data_dev, "test": data_test}

In [25]:
import pickle
pickle.dump(data_train, open("toy_data/train_100k.pkl","wb"))
pickle.dump(data_dev, open("toy_data/dev_100k.pkl","wb"))
pickle.dump(data_test, open("toy_data/test_100k.pkl","wb"))
pickle.dump(data, open("toy_data/data_100k.pkl","wb"))

In [26]:
data_train['x'].shape

torch.Size([100000, 4, 2])

### Describe data

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
z = data_train["z"].numpy()
x = data_train["x"]
y = data_train["y"]

In [ ]:
## Describe z
print ("==Training Data (z)==")

# full-time, flexibile
print ("full-time, flexible", ((z[:,1]==1) & (z[:,2]==1)).sum())
print ("full-time, not flexible",((z[:,1]==1) & (z[:,2]==0)).sum())
print( "not full-time, flexible", ((z[:,1]==0) & (z[:,2]==1)).sum())
print ("not full-time, not flexible",((z[:,1]==0) & (z[:,2]==0)).sum())

# income
plt.hist(z[:,0]*60)
plt.xlabel("Income ($ per hour)")
plt.savefig("toy_data/hist_income.png", dpi=300)

In [ ]:
## Describe z
print ("==Training Data (y)==")

# income 
plt.hist(z[:,0]*60)
plt.xlabel("Income ($ per hour)")
plt.savefig("toy_data/hist_income.png", dpi=300)

In [ ]:
# cost
plt.hist(x[:,1,:].numpy())
plt.xlabel("Cost ($)")
print ("cost (min, max)=", x[:,1,:].min(), x[:,1,:].max())

In [ ]:
l = sorted(x[:,1,:].flatten().tolist())

In [ ]:
# time
plt.hist(x[:,2,:].numpy())
plt.xlabel("Time (min)")
print ("time (min, max)=", x[:,2,:].min(), x[:,2,:].max())

In [ ]:
# waiting time
plt.hist(x[:,2,:].numpy())
plt.xlabel("Waiting Time (min)")
print ("time (min, max)=", x[:,3,:].min(), x[:,3,:].max())

In [ ]:
## Plot true VOT
inc = torch.arange(0.0, 1.0, 0.02).reshape(-1,1)
x_inc = (inc*60).tolist()
n = len(inc)
fulltime = torch.ones(n,1)
nofulltime = torch.zeros(n,1)
flex = torch.ones(n,1)
noflex = torch.zeros(n,1)

In [ ]:
z_full_flex = torch.cat((inc,fulltime,flex),dim=1)
z_full_noflex = torch.cat((inc,fulltime,noflex),dim=1)
z_nofull_flex = torch.cat((inc,nofulltime,flex),dim=1)
z_nofull_noflex = torch.cat((inc,nofulltime,noflex),dim=1)

In [ ]:
true_vots = \
[value_of_x(A0_time, A1_time, A2_time, z_full_flex),
value_of_x(A0_time, A1_time, A2_time, z_full_noflex),
value_of_x(A0_time, A1_time, A2_time, z_nofull_flex),
value_of_x(A0_time, A1_time, A2_time, z_nofull_noflex)]

In [ ]:
true_vowts = \
[value_of_x(A0_wait, A1_wait, A2_wait, z_full_flex),
value_of_x(A0_wait, A1_wait, A2_wait, z_full_noflex),
value_of_x(A0_wait, A1_wait, A2_wait, z_nofull_flex),
value_of_x(A0_wait, A1_wait, A2_wait, z_nofull_noflex)]

In [ ]:
fg_true = plt.figure()
for e in true_vots:
    plt.plot(x_inc,(-e*60).tolist())
plt.legend(['full-time, flex', 'full-time, not flex', 'not full-time, flex', 'not full-time, not flex'])
plt.xlabel("income ($ per hour)")
plt.ylabel("value of time ($ per hour)")
plt.ylim(0, 60)
plt.title("Value of Time vs. Hourly Income (True)")
plt.savefig("toy_data/vot_true.png", dpi=250)

In [ ]:
fg_true = plt.figure()
for e in true_vowts:
    plt.plot(x_inc,(-e*60).tolist())
plt.legend(['full-time, flex', 'full-time, not flex', 'not full-time, flex', 'not full-time, not flex'])
plt.xlabel("income ($ per hour)")
plt.ylabel("value of waiting time ($ per hour)")
plt.ylim(0, 100)
plt.title("Value of Waiting Time vs. Hourly Income (True)")
plt.savefig("toy_data/vowt_true.png", dpi=250)